### Setup/Imports

In [ ]:
import os
import gc
import numpy as np
import torch
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from tqdm import tqdm
import pandas as pd
import scipy.io
from pipeline.paths import Directories, Files
from pipeline.utils import read_scans_agg_file
import scipy.stats
plt.rcParams['figure.dpi'] = 100

### Config

In [ ]:
PHASE = "7"
DATA_VERSION = "13"
SPLIT_TO_ANALYZE = 'VALIDATION'  # Options: 'TRAIN', 'VALIDATION', 'TEST'

MODELS_TO_ANALYZE = [
    {
        'name': 'BBB pi=0.75 mu=0.0 sigma1=1e-1 sigma2=1e-2 beta=1e-2 (10)',
        'type': 'stochastic',
        'domain': 'IMAG',
        'model_version_root': 'MK7_BBB_pi0.75_mu_0.0_sigma1_1e-1_sigma2_1e-3_beta_1e-1',
        'count': 10,
    },
]

# Path to the .pt file containing tumor locations.
# This is a 5D tensor [patient, scan, (x, y, z)]
WORK_ROOT = "D:/NoahSilverberg/ngCBCT"
TUMOR_LOCATIONS_FILE = 'D:/NoahSilverberg/ngCBCT/3D_recon/tumor_location.pt'

# --- Advanced Config ---
SCANS_AGG_FILE = 'scans_to_agg.txt'
SSIM_KWARGS = {"K1": 0.03, "K2": 0.06, "win_size": 15}
SSIM_KWARGS_ = {"k1": 0.03, "k2": 0.06, "kernel_size": 15}

# --- Setup ---
# Create Directories and Files objects
phase_dataver_dir = os.path.join(WORK_ROOT, f"phase{PHASE}", f"DS{DATA_VERSION}")
DIRECTORIES = Directories(
    # projections_results_dir=os.path.join(phase_dataver_dir, "results", "projections"),
    # projections_gated_dir=os.path.join(WORK_ROOT, "gated", "prj_mat"),
    # reconstructions_dir=os.path.join(phase_dataver_dir, "reconstructions"),
    reconstructions_gated_dir=os.path.join(WORK_ROOT, "gated", "fdk_recon"),
    images_results_dir=os.path.join('H:\Public/Noah/phase7/DS13', "results", "images"),
)
FILES = Files(DIRECTORIES)

# Load the list of scans
scans_agg, scan_type_agg = read_scans_agg_file(SCANS_AGG_FILE)
analysis_scans = scans_agg[SPLIT_TO_ANALYZE]

# Load tumor locations
tumor_locations = torch.load(TUMOR_LOCATIONS_FILE, weights_only=False)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE} named '{torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}'")

print(f"\nConfiguration loaded.")
print(f"Analyzing {len(analysis_scans)} scans from the '{SPLIT_TO_ANALYZE}' split.")
print(f"Found {len(MODELS_TO_ANALYZE)} model(s) to analyze.")

## Function definitions, etc.

### Data loading/prep functions

In [ ]:
def load_ground_truth(files_obj: Files, scan_info, domain, slice_idx=None):
    """
    Loads the ground truth data for a given scan and domain.
    """
    patient, scan, scan_type = scan_info
    
    if domain == 'PROJ':
        gt_path = files_obj.get_projections_results_filepath('fdk', patient, scan, scan_type, gated=True)
        data = torch.from_numpy(scipy.io.loadmat(gt_path)['prj']).detach().permute(1, 0, 2)
    elif domain == 'FDK':
        gt_path = files_obj.get_recon_filepath("fdk", patient, scan, scan_type, gated=True, ensure_exists=False)
        data = torch.load(gt_path).detach()
        data = data[20:-20, :, :]
        data = 25. * torch.clip(data, min=0.0, max=0.04)
    elif domain == 'IMAG':
        # The ground truth for the IMAG domain is the FDK of the gated projection
        gt_path = files_obj.get_recon_filepath("fdk", patient, scan, scan_type, gated=True, ensure_exists=False)
        data = torch.load(gt_path).detach()
        data = data[20:-20, :, :]
        data = 25. * torch.clip(data, min=0.0, max=0.04)
    else:
        raise ValueError(f"Unknown domain: {domain}")

    if slice_idx is not None and data.ndim == 3:
        return data[slice_idx]
    return data

def load_predictions(files_obj: Files, model_config, scan_info, slice_idx=None):
    """
    Loads all predictions for a given model, scan, and domain.
    """
    patient, scan, scan_type = scan_info
    domain = model_config['domain']
    root = model_config['model_version_root']
    count = model_config['count']
    model_type = model_config['type']

    predictions = []
    
    print(f"Loading {count} predictions for {model_config['name']}...")
    
    for i in tqdm(range(count), desc="Loading predictions", leave=False):
        passthrough_num = None
        model_version = root

        if model_type == 'ensemble':
            model_version = f"{root}_{i+1:02d}"
        elif model_type == 'stochastic':
            passthrough_num = i

        if domain == 'PROJ':
            pred_path = files_obj.get_projections_results_filepath(model_version, patient, scan, scan_type, gated=False, passthrough_num=passthrough_num, ensure_exists=False)
            pred = torch.from_numpy(scipy.io.loadmat(pred_path)['prj']).detach().permute(1, 0, 2)
        elif domain == 'FDK':
            pred_path = files_obj.get_recon_filepath(model_version, patient, scan, scan_type, gated=False, passthrough_num=passthrough_num, ensure_exists=False)
            pred = torch.load(pred_path).detach()
            pred = pred[20:-20, :, :]
            pred = 25. * torch.clip(pred, min=0.0, max=0.04)
        elif domain == 'IMAG':
            # This assumes the results are saved with the ID model version name
            pred_path = files_obj.get_images_results_filepath(model_version, patient, scan, passthrough_num=passthrough_num, ensure_exists=False)
            pred = torch.load(pred_path).detach()
            pred = torch.squeeze(pred, dim=1)
            pred = torch.permute(pred, (0, 2, 1))
        else:
            raise ValueError(f"Unknown domain: {domain}")
            
        predictions.append(pred)

    predictions_tensor = torch.stack(predictions)
    
    if slice_idx is not None and predictions_tensor.ndim == 4:
        return predictions_tensor[:, slice_idx, :, :]
        
    return predictions_tensor

print("Data loading functions defined.")

### Metric calculation functions: Image quality and pre-calibration uncertainties

In [ ]:
def calculate_ause_sparsification(uncertainty, errors):
    """
    Calculates the Area Under the Sparsification Error curve (AUSE) efficiently.
    """
    uncertainty_flat = uncertainty.flatten()
    errors_flat = errors.flatten()
    
    # Normalize by overall MAE so the curve starts at 1
    overall_mae = np.mean(errors_flat)
    
    def get_sparsification_curve_fast(sorted_errs):
        n_pixels = len(sorted_errs)
        cumulative_errors = np.cumsum(sorted_errs)
        total_error_sum = cumulative_errors[-1]
        sum_errors_removed = np.insert(cumulative_errors[:-1], 0, 0)
        sum_errors_remaining = total_error_sum - sum_errors_removed
        n_remaining = np.arange(n_pixels, 0, -1)
        curve = sum_errors_remaining / n_remaining
        if overall_mae > 0:
            curve = curve / overall_mae # Normalize the curve
        return curve

    # Move arrays to GPU using torch for sorting
    uncertainty_tensor = torch.from_numpy(uncertainty_flat).cuda()
    errors_tensor = torch.from_numpy(errors_flat).cuda()

    # Model curve (sorted by uncertainty)
    model_sorted_indices = torch.argsort(uncertainty_tensor, descending=True)
    model_sorted_errors = errors_tensor[model_sorted_indices].cpu().numpy()
    model_curve = get_sparsification_curve_fast(model_sorted_errors)

    # Oracle curve (sorted by error)
    oracle_sorted_errors = torch.sort(errors_tensor, descending=True)[0].cpu().numpy()
    oracle_curve = get_sparsification_curve_fast(oracle_sorted_errors)
    
    # The AUSE is the area between the two normalized curves
    ause = np.mean(np.abs(model_curve - oracle_curve))
    return ause

def calculate_spearman_correlation(uncertainty, errors, device):
    """
    Calculates the Spearman's Rank Correlation Coefficient between
    the uncertainty and the absolute error.
    """
    uncertainty_flat = torch.from_numpy(uncertainty.flatten()).to(device)
    errors_flat = torch.from_numpy(errors.flatten()).to(device)
    
    # obtain ranks
    xr = torch.argsort(torch.argsort(uncertainty_flat)).float()
    yr = torch.argsort(torch.argsort(errors_flat)).float()

    # demean
    xr = xr - xr.mean()
    yr = yr - yr.mean()

    # compute covariance and norms
    cov = (xr * yr).sum() / (xr.numel() - 1)
    rho = cov / (xr.std(unbiased=True) * yr.std(unbiased=True))
    
    return rho.item()

import torchmetrics
import torchmetrics.image

def calculate_volume_metrics_2_pass(files_obj, model_config, scan_info, gt_volume, device):
    """
    Calculates stats and metrics using a two-pass algorithm for variance for improved stability.
    Also includes a robust PSNR calculation that handles infinite values.
    Pass 1: Calculate the mean of all predictions.
    Pass 2: Calculate variance and other metrics using the pre-calculated mean.
    """
    n_samples = model_config['count']
    gt_volume = gt_volume.to(device) # Ensure GT is on the correct device

    def prediction_generator():
        # This generator yields tensors directly on the GPU
        patient, scan, scan_type = scan_info
        domain = model_config['domain']
        root = model_config['model_version_root']
        model_type = model_config['type']
        for i in range(n_samples):
            passthrough_num = None
            model_version = root
            if model_type == 'ensemble': model_version = f"{root}_{i+1:02d}"
            elif model_type == 'stochastic': passthrough_num = i

            if domain == 'PROJ':
                pred_path = files_obj.get_projections_results_filepath(model_version, patient, scan, scan_type, gated=False, passthrough_num=passthrough_num, ensure_exists=False)
                pred = torch.from_numpy(scipy.io.loadmat(pred_path)['prj']).detach().permute(1, 0, 2)
            elif domain == 'FDK':
                pred_path = files_obj.get_recon_filepath(model_version, patient, scan, scan_type, gated=False, passthrough_num=passthrough_num, ensure_exists=False)
                pred = torch.load(pred_path).detach()
                pred = pred[20:-20, :, :]
                pred = 25. * torch.clip(pred, min=0.0, max=0.04)
            elif domain == 'IMAG':
                pred_path = files_obj.get_images_results_filepath(model_version, patient, scan, passthrough_num=passthrough_num, ensure_exists=False)
                pred = torch.load(pred_path).detach()
                pred = torch.squeeze(pred, dim=1)
                pred = torch.permute(pred, (0, 2, 1))
            yield pred.to(device)

    # --- Pass 1: Calculate Mean ---
    print("Pass 1: Calculating mean prediction...")
    mean_volume = torch.zeros_like(gt_volume)
    # Using a simple sum and divide for the mean
    for pred_volume in tqdm(prediction_generator(), total=n_samples, desc="Pass 1/2 (Mean)", leave=False):
        mean_volume += pred_volume
    mean_volume /= n_samples

    # --- Pass 2: Calculate Variance and Metrics ---
    print("Pass 2: Calculating variance and metrics...")
    sum_sq_diff_volume = torch.zeros_like(gt_volume)
    sample_avg_ssims, sample_avg_psnrs, sample_avg_mses, sample_avg_maes = [], [], [], []

    # Initialize metrics on the specified device
    data_range = gt_volume.max() - gt_volume.min()
    ssim_metric = torchmetrics.image.StructuralSimilarityIndexMeasure(data_range=data_range, **SSIM_KWARGS_).to(device)

    # PSNR metric that returns per-slice results to handle 'inf'
    psnr_metric = torchmetrics.image.PeakSignalNoiseRatio(data_range=data_range, reduction='none').to(device)

    for pred_volume in tqdm(prediction_generator(), total=n_samples, desc="Pass 2/2 (Var & Metrics)", leave=False):
        # Variance calculation
        diff = pred_volume - mean_volume
        sum_sq_diff_volume += diff * diff

        # Per-sample metrics
        if gt_volume.ndim > 2:
            pred_vol_batch = pred_volume.unsqueeze(1)
            gt_vol_batch = gt_volume.unsqueeze(1)
            
            sample_avg_ssims.append(ssim_metric(pred_vol_batch, gt_vol_batch).item())
            sample_avg_mses.append(torch.mean((gt_volume - pred_volume)**2).item())
            sample_avg_maes.append(torch.mean(torch.abs(gt_volume - pred_volume)).item())

            # --- Robust PSNR Calculation ---
            psnr_per_slice = psnr_metric(pred_vol_batch, gt_vol_batch)
            finite_psnrs = psnr_per_slice[torch.isfinite(psnr_per_slice)] # Filter out inf values
            if finite_psnrs.numel() > 0:
                sample_avg_psnrs.append(torch.mean(finite_psnrs).item())
            else:
                # For debugging, print abs difference between GT and prediction
                abs_diff = torch.sum(torch.abs(gt_volume - pred_volume))
                print(f"Absolute difference between GT and prediction: {abs_diff.item()}")
                # Raise error
                raise ValueError("All slices in the prediction are perfect matches, leading to infinite PSNR values.")

    # Finalize variance and uncertainty
    if n_samples > 1:
        # Using n_samples for population standard deviation, as in the original code.
        # For sample standard deviation, use (n_samples - 1).
        variance_volume_map = sum_sq_diff_volume / n_samples
        uncertainty_volume_map = torch.sqrt(variance_volume_map)
    else:
        uncertainty_volume_map = torch.zeros_like(mean_volume)

    # --- Calculate metrics for the mean prediction ---
    metrics = {}
    if gt_volume.ndim > 2:
        mean_vol_batch = mean_volume.unsqueeze(1)
        gt_vol_batch = gt_volume.unsqueeze(1)
        
        metrics['mean_ssim'] = ssim_metric(mean_vol_batch, gt_vol_batch).item()
        metrics['mean_mse'] = torch.mean((gt_volume - mean_volume)**2).item()
        metrics['mean_mae'] = torch.mean(torch.abs(gt_volume - mean_volume)).item()

        # Robust PSNR for the mean prediction
        mean_psnr_per_slice = psnr_metric(mean_vol_batch, gt_vol_batch)
        finite_mean_psnrs = mean_psnr_per_slice[torch.isfinite(mean_psnr_per_slice)]
        if finite_mean_psnrs.numel() > 0:
            metrics['mean_psnr'] = torch.mean(finite_mean_psnrs).item()
        else:
            # For debugging, print abs difference between GT and mean prediction
            abs_diff = torch.sum(torch.abs(gt_volume - mean_volume))
            print(f"Absolute difference between GT and mean prediction: {abs_diff.item()}")
            # Raise error
            raise ValueError("All slices in the mean prediction are perfect matches, leading to infinite PSNR values.")

    # --- Aggregate per-sample metrics ---
    metrics['sample_avg_ssim'] = np.mean(sample_avg_ssims) if sample_avg_ssims else 0
    metrics['sample_avg_psnr'] = np.mean(sample_avg_psnrs) if sample_avg_psnrs else 0
    metrics['sample_avg_mse'] = np.mean(sample_avg_mses) if sample_avg_mses else 0
    metrics['sample_avg_mae'] = np.mean(sample_avg_maes) if sample_avg_maes else 0

    return metrics, mean_volume, uncertainty_volume_map

print("Metric calculation functions defined.")

### Uncertainty calibration and post-calibration metric functions

In [ ]:
import numpy as np
import scipy.stats
from scipy.interpolate import interp1d
from scipy.optimize import isotonic_regression

# --- Calibration Methods ---

def calculate_platt_scaler(errors, std_devs):
    """
    Calculates the optimal scaling factor 'T' for variance scaling.
    This factor is found by minimizing the NLL on a validation set.
    The optimal T is sqrt(mean(squared_error / variance)).

    Args:
        errors (np.ndarray): The absolute errors (ground_truth - prediction).
        std_devs (np.ndarray): The predicted standard deviations.

    Returns:
        float: The scaling factor T.
    """
    errors_flat = errors.flatten()
    std_devs_flat = std_devs.flatten()

    # To avoid division by zero, add a small epsilon to the variance
    variances_flat = std_devs_flat**2 + 1e-6

    # Calculate T^2 = mean(error^2 / variance)
    t_squared = np.mean(errors_flat**2 / variances_flat)

    return np.sqrt(t_squared)


def train_isotonic_regression(predicted_std, observed_errors, device):
    """
    Trains an Isotonic Regression model using scipy.optimize.isotonic_regression.
    """
    pred_std_flat = predicted_std.flatten()
    obs_err_flat = observed_errors.flatten()

    # Sort by predicted standard deviation
    print("Sorting erorrs...")
    sort_indices = torch.argsort(torch.from_numpy(pred_std_flat).to(device)).cpu().numpy()
    sorted_pred_std = pred_std_flat[sort_indices]
    sorted_obs_err_sq = obs_err_flat[sort_indices]**2

    # Apply scipy's isotonic regression
    print("Applying isotonic regression...")
    calibrated_variances = isotonic_regression(sorted_obs_err_sq).x
    calibrated_std = np.sqrt(calibrated_variances)

    # Create an interpolation function to map new predictions
    unique_pred_std, unique_indices = np.unique(sorted_pred_std, return_index=True)
    unique_calib_std = calibrated_std[unique_indices]
    
    # check that the unique pred std are sorted
    if not np.all(np.diff(unique_pred_std) >= 0):
        raise ValueError("Predicted standard deviations are not sorted. Ensure the input is sorted before applying isotonic regression.")

    print("Creating interpolation model...")
    iso_model = interp1d(unique_pred_std, unique_calib_std, kind='linear', bounds_error=False, 
                         fill_value=(unique_calib_std[0], unique_calib_std[-1]), assume_sorted=True)

    return iso_model


# --- New Evaluation Metrics ---

def calculate_nll(ground_truth, mean_pred, uncertainty_map):
    """
    Calculates the Negative Log-Likelihood (NLL) for a Gaussian prediction.
    """
    gt_flat = ground_truth.flatten()
    pred_flat = mean_pred.flatten()
    uncert_flat = uncertainty_map.flatten()

    # Add a small epsilon to variance to prevent log(0) or division by zero
    variance = uncert_flat**2 + 1e-9
    
    # NLL formula for a Gaussian distribution
    nll_values = 0.5 * (np.log(2 * np.pi * variance) + (gt_flat - pred_flat)**2 / variance)
    
    return np.mean(nll_values)


def calculate_all_eces(ground_truth, mean_pred, uncertainty_map, n_bins=20):
    """
    Calculates variations of the Expected Calibration Error (ECE) for regression.
    """
    gt_flat = ground_truth.flatten()
    pred_flat = mean_pred.flatten()
    uncert_flat = uncertainty_map.flatten() + 1e-9 # Avoid zero std dev

    # Get the predicted CDF value for each ground truth point
    pred_cdfs = scipy.stats.norm.cdf(gt_flat, loc=pred_flat, scale=uncert_flat)
    
    expected_confidence_levels = np.linspace(0, 1, n_bins)
    observed_frequencies = np.array([np.mean(pred_cdfs <= p_j) for p_j in expected_confidence_levels])

    # --- Calculate bin weights (proportional to number of points in each bin) ---
    bin_weights = np.zeros(n_bins)
    for i in range(1, n_bins):
        lower_bound = expected_confidence_levels[i-1]
        upper_bound = expected_confidence_levels[i]
        points_in_bin = (pred_cdfs > lower_bound) & (pred_cdfs <= upper_bound)
        bin_weights[i] = np.mean(points_in_bin)
    
    if np.sum(bin_weights) > 0:
        bin_weights /= np.sum(bin_weights)
    
    # --- Calculate ECE variants ---
    abs_diff = np.abs(expected_confidence_levels - observed_frequencies)
    sq_diff = (expected_confidence_levels - observed_frequencies)**2

    ece_weighted_abs = np.sum(bin_weights * abs_diff)
    ece_weighted_sq = np.sum(bin_weights * sq_diff)
    ece_unweighted_abs = np.mean(abs_diff)
    ece_unweighted_sq = np.mean(sq_diff)
    
    return {
        'ece_weighted_abs': ece_weighted_abs,
        'ece_weighted_sq': ece_weighted_sq,
        'ece_unweighted_abs': ece_unweighted_abs,
        'ece_unweighted_sq': ece_unweighted_sq,
    }


def calculate_ence(ground_truth, mean_pred, uncertainty_map, device, n_bins=20):
    """
    Calculates the Expected Normalized Calibration Error (ENCE) using quantile-based binning
    as described in Levi et al., 2020.
    """
    gt_flat = ground_truth.flatten()
    pred_flat = mean_pred.flatten()
    uncert_flat = uncertainty_map.flatten()
    
    # Ensure there are enough unique values for binning
    if len(np.unique(uncert_flat)) < n_bins:
        print(f"Warning: Number of unique uncertainties is less than n_bins. ENCE may be unreliable.")

    # Get the indices that would sort the uncertainties
    sorted_indices = torch.argsort(torch.from_numpy(uncert_flat).to(device)).cpu().numpy()
    
    # Split the sorted indices into N bins of equal size.
    # np.array_split handles cases where the total number of points is not divisible by n_bins.
    binned_indices = np.array_split(sorted_indices, n_bins)

    ence_sum = 0
    
    # Loop through each bin of indices
    for bin_idx_list in binned_indices:
        # Skip empty bins, though this is unlikely with quantile binning
        if len(bin_idx_list) > 0:
            # Root Mean Variance (RMV) in bin
            rmv_j = np.sqrt(np.mean(uncert_flat[bin_idx_list]**2)) + 1e-9
            
            # Root Mean Squared Error (RMSE) in bin
            rmse_j = np.sqrt(np.mean((gt_flat[bin_idx_list] - pred_flat[bin_idx_list])**2))

            # Add the normalized error for this bin to the sum
            ence_sum += np.abs(rmv_j - rmse_j) / rmv_j

    return ence_sum / n_bins


def calculate_mpiw(uncertainty_map, confidence_levels=[0.68, 0.95]):
    """
    Calculates the Mean Prediction Interval Width (MPIW) for given confidence levels.
    """
    uncert_flat = uncertainty_map.flatten()
    results = {}
    for level in confidence_levels:
        # Get the z-score for the confidence level (e.g., 1.96 for 95%)
        z_score = scipy.stats.norm.ppf(1 - (1 - level) / 2)
        
        # Width of the prediction interval
        widths = 2 * z_score * uncert_flat
        
        # Store the mean width
        results[f'mpiw_{int(level*100)}'] = np.mean(widths)
        
    return results

print("✅ Calibration and advanced uncertainty metric functions are defined.")

### Visualization functions

In [ ]:
def plot_mean_comparison(mean_pred, ground_truth, uncertainty_map, model_name, scan_name, slice_idx, tumor_coords_xy=None):
    """Plots the GT, mean prediction, absolute error, and uncertainty map."""
    error_map = np.abs(ground_truth - mean_pred)

    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    fig.suptitle(f'{model_name} - {scan_name} - Slice {slice_idx} (Mean vs. GT)', fontsize=16)

    im1 = axes[0].imshow(ground_truth, cmap='gray')
    axes[0].set_title('Ground Truth')
    axes[0].axis('off')
    fig.colorbar(im1, ax=axes[0])

    if tumor_coords_xy:
        x, y = tumor_coords_xy
        for i in range(4):
            axes[i].annotate('', xy=(x, y), xytext=(x - 30, y - 30),
                            arrowprops=dict(facecolor='red', edgecolor='red', shrink=0.05, width=1, headwidth=5, headlength=5))

    im2 = axes[1].imshow(mean_pred, cmap='gray')
    axes[1].set_title('Mean Prediction')
    axes[1].axis('off')
    fig.colorbar(im2, ax=axes[1])

    im3 = axes[2].imshow(error_map, cmap='magma')
    axes[2].set_title('Absolute Error Map')
    axes[2].axis('off')
    fig.colorbar(im3, ax=axes[2])

    im4 = axes[3].imshow(uncertainty_map, cmap='viridis')
    axes[3].set_title('Uncertainty (Std Dev)')
    axes[3].axis('off')
    fig.colorbar(im4, ax=axes[3])

    plt.tight_layout()
    plt.show()

def plot_ssim_map(ssim_map, model_name, scan_name):
    """Plots the SSIM map."""
    plt.figure(figsize=(3, 3))
    plt.imshow(ssim_map, cmap='viridis', vmin=0, vmax=1)
    plt.title(f'SSIM Map - {model_name} - {scan_name}')
    plt.colorbar()
    plt.axis('off')
    plt.show()

def plot_calibration_curve(ground_truth, mean_pred, uncertainty_map, model_name, scan_name, n_levels=20):
    """
    Plots the calibration curve with a marginal histogram below the x-axis
    showing the distribution of the predicted CDF values.
    """
    gt_flat = ground_truth.flatten()
    pred_flat = mean_pred.flatten()
    uncert_flat = uncertainty_map.flatten()

    # Calculate the predicted CDF value for each point
    pred_cdfs = scipy.stats.norm.cdf(gt_flat, loc=pred_flat, scale=uncert_flat)
    
    # --- Create figure with two subplots, sharing the x-axis ---
    fig, (ax_cal, ax_hist) = plt.subplots(
        2, 1,
        figsize=(8, 8),
        sharex=True,
        gridspec_kw={'height_ratios': [3, 1]} # Main plot is 3x taller
    )
    
    # --- Main Calibration Plot (top) ---
    expected_confidence_levels = np.linspace(0, 1, n_levels)
    observed_frequencies = np.array([np.mean(pred_cdfs <= p_j) for p_j in expected_confidence_levels])

    ax_cal.plot([0, 1], [0, 1], '--', color='grey', label='Perfectly Calibrated')
    ax_cal.plot(expected_confidence_levels, observed_frequencies, '-o', label='Model Calibration')
    ax_cal.set_ylabel('Observed Confidence Level')
    ax_cal.set_title(f'Calibration Plot - {model_name} - {scan_name}')
    ax_cal.legend()
    ax_cal.grid(True, linestyle=':')

    # --- Marginal Histogram (bottom) ---
    ax_hist.hist(pred_cdfs, bins=50, range=(0,1), density=True, color='steelblue', alpha=0.8)
    ax_hist.set_xlabel('Expected Confidence Level (Predicted CDF)')
    ax_hist.set_ylabel('Density')
    ax_hist.set_yscale('log')
    # ax_hist.set_yticks([]) # Hide y-ticks for clarity

    # Final adjustments
    plt.tight_layout()
    plt.show()

def plot_sparsification_curve(uncertainty, errors, model_name, scan_name, device):
    """
    Plots the model and oracle sparsification curves used for AUSE calculation.
    """
    uncertainty_flat = uncertainty.flatten()
    errors_flat = errors.flatten()
    
    def get_sparsification_curve_fast(sorted_errs, overall_mae):
        n_pixels = len(sorted_errs)
        cumulative_errors = np.cumsum(sorted_errs)
        total_error_sum = cumulative_errors[-1]
        sum_errors_removed = np.insert(cumulative_errors[:-1], 0, 0)
        sum_errors_remaining = total_error_sum - sum_errors_removed
        n_remaining = np.arange(n_pixels, 0, -1)
        curve = sum_errors_remaining / n_remaining
        if overall_mae > 0:
            curve = curve / overall_mae
        return curve
    
    overall_mae = np.mean(errors_flat)

    # Model curve (sorted by uncertainty)
    model_sorted_indices = torch.argsort(torch.from_numpy(uncertainty_flat).to(device), descending=True).cpu().numpy()
    model_sorted_errors = errors_flat[model_sorted_indices]
    model_curve = get_sparsification_curve_fast(model_sorted_errors, overall_mae)

    # Oracle curve (sorted by error)
    oracle_sorted_errors = torch.sort(torch.from_numpy(errors_flat).to(device), descending=True).values.cpu().numpy()
    oracle_curve = get_sparsification_curve_fast(oracle_sorted_errors, overall_mae)
    
    # X-axis: fraction of pixels removed
    fraction_removed = np.linspace(0, 1, len(model_curve))

    # Downsample to 1000 points
    if len(fraction_removed) > 1000:
        step = len(fraction_removed) // 1000
        fraction_removed = fraction_removed[::step]
        model_curve = model_curve[::step]
        oracle_curve = oracle_curve[::step]
    
    plt.figure(figsize=(7, 6))
    plt.plot(fraction_removed, model_curve, label='Model (Sort by Uncertainty)')
    plt.plot(fraction_removed, oracle_curve, '--', label='Oracle (Sort by Error)')
    plt.xlabel('Fraction of Pixels Removed')
    plt.ylabel('Mean Absolute Error of Remaining Pixels')
    plt.title(f'Sparsification Curve - {model_name} - {scan_name}')
    plt.legend()
    plt.grid(True, linestyle=':')
    plt.show()

def plot_samples_comparison(ground_truth, mean_pred, samples, model_name, scan_name, slice_idx, tumor_coords_xy=None):
    """
    Plots the ground truth, mean prediction, and a few individual sample predictions.
    
    Args:
        ground_truth (np.ndarray): The 2D ground truth slice.
        mean_pred (np.ndarray): The 2D mean prediction slice.
        samples (list of np.ndarray): A list of 2D sample prediction slices.
        model_name (str): The name of the model for the title.
        scan_name (str): The name of the scan for the title.
        slice_idx (int): The index of the slice for the title.
        tumor_coords_xy (tuple, optional): (x, y) coordinates for the tumor arrow.
    """
    num_samples = len(samples)
    # Total columns = 1 for GT + 1 for Mean + N for samples
    num_cols = 2 + num_samples
    
    fig, axes = plt.subplots(1, num_cols, figsize=(4 * num_cols, 4.5), constrained_layout=True)
    fig.suptitle(f'{model_name} - {scan_name} - Slice {slice_idx} (GT, Mean, and Samples)', fontsize=16)

    # Determine a consistent grayscale range based on the ground truth and mean
    vmin = min(ground_truth.min(), mean_pred.min())
    vmax = max(ground_truth.max(), mean_pred.max())

    # --- Plot Ground Truth ---
    axes[0].imshow(ground_truth, cmap='gray', vmin=vmin, vmax=vmax)
    axes[0].set_title('Ground Truth')
    axes[0].axis('off')

    # --- Plot Mean Prediction ---
    axes[1].imshow(mean_pred, cmap='gray', vmin=vmin, vmax=vmax)
    axes[1].set_title('Mean Prediction')
    axes[1].axis('off')

    # --- Plot Samples ---
    for i in range(num_samples):
        ax = axes[i + 2]
        im = ax.imshow(samples[i], cmap='gray', vmin=vmin, vmax=vmax)
        ax.set_title(f'Sample {i+1}')
        ax.axis('off')

    # --- Add Tumor Arrow ---
    if tumor_coords_xy:
        x, y = tumor_coords_xy
        for ax in axes:
            ax.annotate('', xy=(x, y), xytext=(x - 30, y - 30),
                        arrowprops=dict(facecolor='red', edgecolor='red', shrink=0.05, 
                                        width=1, headwidth=5, headlength=5))

    plt.show()

def plot_worst_samples_comparison(ground_truth, mean_pred, worst_samples_data, model_name, scan_name, slice_idx, tumor_coords_xy=None):
    """
    Plots the ground truth, mean prediction, and the worst-performing sample predictions.
    
    Args:
        ground_truth (np.ndarray): The 2D ground truth slice.
        mean_pred (np.ndarray): The 2D mean prediction slice.
        worst_samples_data (list): A list of tuples, where each tuple is 
                                   (loss, sample_slice_numpy, sample_index).
        model_name (str): The name of the model for the title.
        scan_name (str): The name of the scan for the title.
        slice_idx (int): The index of the slice for the title.
        tumor_coords_xy (tuple, optional): (x, y) coordinates for the tumor arrow.
    """
    num_samples = len(worst_samples_data)
    num_cols = 2 + num_samples
    
    fig, axes = plt.subplots(1, num_cols, figsize=(4 * num_cols, 5), constrained_layout=True)
    fig.suptitle(f'{model_name} - {scan_name} - Slice {slice_idx} (Top {num_samples} Worst Samples by SmoothL1Loss)', fontsize=16)

    # Determine a consistent grayscale range
    vmin = min(ground_truth.min(), mean_pred.min())
    vmax = max(ground_truth.max(), mean_pred.max())

    # --- Plot Ground Truth ---
    axes[0].imshow(ground_truth, cmap='gray', vmin=vmin, vmax=vmax)
    axes[0].set_title('Ground Truth')
    axes[0].axis('off')

    # --- Plot Mean Prediction ---
    axes[1].imshow(mean_pred, cmap='gray', vmin=vmin, vmax=vmax)
    axes[1].set_title('Mean Prediction')
    axes[1].axis('off')

    # --- Plot Worst Samples ---
    for i in range(num_samples):
        loss, sample_slice, sample_idx = worst_samples_data[i]
        ax = axes[i + 2]
        im = ax.imshow(sample_slice, cmap='gray', vmin=vmin, vmax=vmax)
        # Add the loss and original sample number to the title
        ax.set_title(f'Sample #{sample_idx}\nLoss: {loss:.4f}')
        ax.axis('off')

    # --- Add Tumor Arrow ---
    if tumor_coords_xy:
        x, y = tumor_coords_xy
        for ax in axes:
            ax.annotate('', xy=(x, y), xytext=(x - 30, y - 30),
                        arrowprops=dict(facecolor='red', edgecolor='red', shrink=0.05, 
                                        width=1, headwidth=5, headlength=5))

    plt.show()

def plot_combined_calibration_curves(
    ground_truth, mean_pred, 
    platt_uncertainty_map, iso_uncertainty_map, 
    model_name, scan_name, n_bins=20
):
    """
    Plots ECE and ENCE calibration curves on a single figure,
    comparing STD Scaling and Isotonic Regression directly.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
    fig.suptitle(f'Calibration Comparison for {model_name} on {scan_name}', fontsize=16)

    # --- ECE Subplot (ax1) ---
    ece_calibrations = {'STD Scaling': platt_uncertainty_map, 'Isotonic': iso_uncertainty_map}
    ece_values = {}

    for name, uncert_map in ece_calibrations.items():
        gt_flat, pred_flat, uncert_flat = ground_truth.flatten(), mean_pred.flatten(), uncert_map.flatten() + 1e-9
        pred_cdfs = scipy.stats.norm.cdf(gt_flat, loc=pred_flat, scale=uncert_flat)
        
        expected_confidence = np.linspace(0, 1, n_bins + 1)
        observed_confidence = np.array([np.mean(pred_cdfs <= p_j) for p_j in expected_confidence])
        
        ax1.plot(expected_confidence, observed_confidence, '-o', label=name, alpha=0.8)
        
        ece_metric = calculate_all_eces(ground_truth, mean_pred, uncert_map, n_bins=n_bins)
        ece_values[name] = ece_metric['ece_unweighted_abs']

    ax1.plot([0, 1], [0, 1], '--', color='grey', label='Perfect')
    ax1.set_title(f'ECE Plot | STD ECE: {ece_values["STD Scaling"]:.4f} | Isotonic ECE: {ece_values["Isotonic"]:.4f}')
    ax1.set_xlabel('Expected Confidence Level')
    ax1.set_ylabel('Observed Confidence Level')
    ax1.legend()
    ax1.grid(True, linestyle=':')
    ax1.axis('equal')
    ax1.set_xlim([0, 1])
    ax1.set_ylim([0, 1])

    # --- ENCE Subplot (ax2) ---
    ence_calibrations = {'STD Scaling': platt_uncertainty_map, 'Isotonic': iso_uncertainty_map}
    ence_values = {}
    all_points = []

    for name, uncert_map in ence_calibrations.items():
        gt_flat, pred_flat, uncert_flat = ground_truth.flatten(), mean_pred.flatten(), uncert_map.flatten()
        
        sorted_indices = torch.argsort(torch.from_numpy(uncert_flat).to(DEVICE)).cpu().numpy()
        binned_indices = np.array_split(sorted_indices, n_bins)
        
        bin_rmv, bin_rmse = [], []
        for bin_idx_list in binned_indices:
            if len(bin_idx_list) > 0:
                rmv_j = np.sqrt(np.mean(uncert_flat[bin_idx_list]**2))
                rmse_j = np.sqrt(np.mean((gt_flat[bin_idx_list] - pred_flat[bin_idx_list])**2))
                bin_rmv.append(rmv_j)
                bin_rmse.append(rmse_j)
        
        ax2.plot(bin_rmv, bin_rmse, '-o', label=name, alpha=0.8, zorder=3)
        all_points.extend(bin_rmv)
        all_points.extend(bin_rmse)
        
        ence_values[name] = calculate_ence(ground_truth, mean_pred, uncert_map, DEVICE, n_bins=n_bins)

    if all_points:
      max_val = np.max(all_points) * 1.1
      ax2.plot([0, max_val], [0, max_val], '--', color='grey', label='Perfect')
      ax2.set_xlim(left=0, right=max_val)
      ax2.set_ylim(bottom=0, top=max_val)

    ax2.set_title(f'ENCE Plot (RMSE vs. RMV) | STD ENCE: {ence_values["STD Scaling"]:.4f} | Isotonic ENCE: {ence_values["Isotonic"]:.4f}')
    ax2.set_xlabel('Root Mean Variance (RMV) per Bin')
    ax2.set_ylabel('Root Mean Squared Error (RMSE) per Bin')
    ax2.legend()
    ax2.grid(True, linestyle=':')
    ax2.axis('equal')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

print("Visualization functions defined.")

## Pre-calibration processing & results

NOTES FOR BBB:
pi=0.5 has been bad for all (hallucinations)
pi=0.75, sigma1=1e-1, sigma2=1e-3, beta=1e-2 was good
pi=0.25, sigma1=1e-1, sigma2=1e-3, beta=1e-2 was good
pi=0.75, sigma1=1e-1, sigma2=1e-3, beta=1e-3 had hallucinations
pi=0.75, beta=1e-2, and sigma1=5e-1 or sigma2=1e-2 hallucinated

### Main loop: Pre-calibration

In [ ]:
# This list will store dictionaries of results for each scan and model
all_results = []

for model_config in MODELS_TO_ANALYZE:
    model_name = model_config['name']
    domain = model_config['domain']
    
    scan_results = []

    for scan_info in tqdm(analysis_scans, desc=f"Analyzing Model: {model_name}"):
        patient, scan, _ = scan_info
        scan_name = f"p{patient}_{scan}"
        
        # --- Determine Domain and Plotting Slice ---
        is_visual_domain = domain in ['FDK', 'IMAG']
        plot_slice_idx = None
        tumor_xy = None
        
        if is_visual_domain:
            if 'tumor_locations' in locals() and tumor_locations is not None:
                try:
                    loc = tumor_locations[int(patient), int(scan)]
                    tumor_xy = (loc[1].item(), loc[0].item())
                    plot_slice_idx = int(loc[2].item()) - 20
                except (IndexError, TypeError):
                    raise Exception(f"Could not find tumor location for {scan_name}.")
            else:
                raise Exception(f"Could not find tumor location for {scan_name}.")
        
        # --- Data Loading (Ground Truth Only) ---
        gt_volume = load_ground_truth(FILES, scan_info, domain, slice_idx=None)
        gt_volume_np = gt_volume.cpu().numpy()
        
        # --- Iterative Metric Calculation ---
        print("Calculating metrics iteratively...")
        iq_metrics, mean_pred_vol, uncertainty_map_vol = calculate_volume_metrics_2_pass(
            FILES, model_config, scan_info, gt_volume.to(DEVICE), DEVICE
        )
        mean_pred_vol = mean_pred_vol.cpu().numpy()
        uncertainty_map_vol = uncertainty_map_vol.cpu().numpy()

        
        # --- Uncertainty Metric Calculation ---
        errors_vol = np.abs(gt_volume_np - mean_pred_vol)
        print("Calculating AUSE...")
        ause_val = calculate_ause_sparsification(uncertainty_map_vol, errors_vol)
        # print("Calculating ECE...")
        # ece_val = calculate_ece(gt_volume_np, mean_pred_vol, uncertainty_map_vol)
        print("Calculating Spearman's correlation...")
        spearman_val = calculate_spearman_correlation(uncertainty_map_vol, errors_vol, DEVICE)
        
        # --- Store Results ---
        scan_result = {
            'model_name': model_name,
            'scan_name': scan_name,
            **iq_metrics,
            'ause': ause_val,
            # 'ece': ece_val,
            'spearman_corr': spearman_val,
        }
        scan_results.append(scan_result)
        
        # --- Visualization ---
        print(f"\n--- Results for {model_name} on {scan_name} ---")
        
        # plot_calibration_curve(gt_volume_np, mean_pred_vol, uncertainty_map_vol, model_name, scan_name)
        # plot_sparsification_curve(uncertainty_map_vol, errors_vol, model_name, scan_name, DEVICE)
        
        if is_visual_domain:
            gt_slice_np = gt_volume_np[plot_slice_idx]
            mean_pred_slice = mean_pred_vol[plot_slice_idx]
            uncertainty_map_slice = uncertainty_map_vol[plot_slice_idx]
            _, ssim_map_val = ssim(gt_slice_np, mean_pred_slice, 
                                  data_range=(np.max(gt_slice_np) - np.min(gt_slice_np)), 
                                  full=True, **SSIM_KWARGS)
            
            plot_mean_comparison(mean_pred_slice, gt_slice_np, uncertainty_map_slice, 
                                 model_name, scan_name, plot_slice_idx, tumor_coords_xy=tumor_xy)
            # plot_ssim_map(ssim_map_val, model_name, scan_name)

            # --- Load and plot a few samples for visual comparison ---
            print("Loading and plotting samples...")
            SAMPLES_TO_PLOT = 3
            sample_slices_for_plotting = []
            
            # Determine the number of samples to load (can't be more than what's available)
            num_to_load = min(SAMPLES_TO_PLOT, model_config['count'])

            for i in range(num_to_load):
                passthrough_num = None
                model_version = model_config['model_version_root']
                model_type = model_config['type']

                if model_type == 'ensemble':
                    model_version = f"{model_config['model_version_root']}_{i+1:02d}"
                elif model_type == 'stochastic':
                    passthrough_num = i
                
                # This loading logic is copied from your metrics function
                pred = None
                if domain == 'FDK':
                    pred_path = FILES.get_recon_filepath(model_version, patient, scan, scan_type_agg, gated=False, passthrough_num=passthrough_num, ensure_exists=False)
                    pred = torch.load(pred_path).detach()
                    pred = pred[20:-20, :, :]
                    pred = 25. * torch.clip(pred, min=0.0, max=0.04)
                elif domain == 'IMAG':
                    pred_path = FILES.get_images_results_filepath(model_version, patient, scan, passthrough_num=passthrough_num, ensure_exists=False)
                    pred = torch.load(pred_path).detach()
                    pred = torch.squeeze(pred, dim=1)
                    pred = torch.permute(pred, (0, 2, 1))
                
                # Get the specific slice and convert to a numpy array for plotting
                if pred is not None:
                    sample_slice = pred[plot_slice_idx].cpu().numpy()
                    sample_slices_for_plotting.append(sample_slice)

            # Call the new plotting function
            if sample_slices_for_plotting:
                plot_samples_comparison(
                    ground_truth=gt_slice_np,
                    mean_pred=mean_pred_slice,
                    samples=sample_slices_for_plotting,
                    model_name=model_name,
                    scan_name=scan_name,
                    slice_idx=plot_slice_idx,
                    tumor_coords_xy=tumor_xy
                )

            # print("Finding and plotting worst samples by Smooth L1 Loss...")
            # WORST_SAMPLES_TO_PLOT = 5

            # # List to store tuples of (loss, sample_slice_numpy, sample_index)
            # worst_samples_data = []
            # # Get the ground truth slice as a tensor on the correct device
            # gt_slice_tensor = gt_volume[plot_slice_idx].to(DEVICE)

            # # Loop through all available samples to find the worst ones
            # for i in tqdm(range(model_config['count']), desc="Finding Worst Samples", leave=False):
            #     passthrough_num = None
            #     model_version = model_config['model_version_root']
            #     model_type = model_config['type']

            #     if model_type == 'ensemble':
            #         model_version = f"{model_config['model_version_root']}_{i+1:02d}"
            #     elif model_type == 'stochastic':
            #         passthrough_num = i
                
            #     # Load the prediction volume as a tensor on the GPU
            #     pred_vol_tensor = None
            #     if domain == 'FDK':
            #         pred_path = FILES.get_recon_filepath(model_version, patient, scan, scan_type_agg, gated=False, passthrough_num=passthrough_num, ensure_exists=False)
            #         pred_vol_tensor = torch.load(pred_path, map_location=DEVICE).detach()
            #         pred_vol_tensor = pred_vol_tensor[20:-20, :, :]
            #         pred_vol_tensor = 25. * torch.clip(pred_vol_tensor, min=0.0, max=0.04)
            #     elif domain == 'IMAG':
            #         pred_path = FILES.get_images_results_filepath(model_version, patient, scan, passthrough_num=passthrough_num, ensure_exists=False)
            #         pred_vol_tensor = torch.load(pred_path, map_location=DEVICE).detach()
            #         pred_vol_tensor = torch.squeeze(pred_vol_tensor, dim=1)
            #         pred_vol_tensor = torch.permute(pred_vol_tensor, (0, 2, 1))

            #     if pred_vol_tensor is not None:
            #         pred_slice_tensor = pred_vol_tensor[plot_slice_idx]
                    
            #         # Calculate Smooth L1 Loss for the current slice
            #         import torch.nn.functional as F
            #         loss = F.smooth_l1_loss(pred_slice_tensor, gt_slice_tensor, reduction='mean').item()

            #         # Keep track of the top 5 worst samples (highest loss)
            #         if len(worst_samples_data) < WORST_SAMPLES_TO_PLOT:
            #             worst_samples_data.append((loss, pred_slice_tensor.cpu().numpy(), i))
            #         else:
            #             # Find the sample with the minimum loss currently in our list
            #             min_loss_in_list = min(worst_samples_data, key=lambda x: x[0])
            #             if loss > min_loss_in_list[0]:
            #                 # If current sample is worse, replace the "best of the worst"
            #                 worst_samples_data.remove(min_loss_in_list)
            #                 worst_samples_data.append((loss, pred_slice_tensor.cpu().numpy(), i))

            # # Sort the final list from worst to best for plotting
            # worst_samples_data.sort(key=lambda x: x[0], reverse=True)

            # # Call the new plotting function with the results
            # if worst_samples_data:
            #     plot_worst_samples_comparison(
            #         ground_truth=gt_slice_np,
            #         mean_pred=mean_pred_slice,
            #         worst_samples_data=worst_samples_data,
            #         model_name=model_name,
            #         scan_name=scan_name,
            #         slice_idx=plot_slice_idx,
            #         tumor_coords_xy=tumor_xy
            #     )

        # --- Clean up memory ---
        del gt_volume, gt_volume_np, mean_pred_vol, uncertainty_map_vol, errors_vol
        gc.collect()
        
    all_results.extend(scan_results)

# Convert results to a pandas DataFrame for easier analysis
results_df = pd.DataFrame(all_results)

print("\n\n✅ Analysis complete for all models and scans.")
results_df

### Summary/Comparison

In [ ]:
# results_df = pd.read_csv('BBB_results_val.csv')

# Only aggregate numeric columns
numeric_cols = results_df.select_dtypes(include=[np.number]).columns
summary = results_df.groupby('model_name')[numeric_cols].agg(['mean', 'std']).reset_index()

# save results_df and summary to CSV files
results_df.to_csv('BBB_results_val.csv', index=False)
summary.to_csv('BBB_summary_val.csv', index=False)

# Prepare summary_display with formatted mean ± std for each metric
summary_display = pd.DataFrame()
summary_display['model_name'] = summary['model_name']
for col in numeric_cols:
    mean_col = (col, 'mean')
    std_col = (col, 'std')
    summary_display[col] = summary[mean_col].map('{:.4f}'.format) + ' ± ' + summary[std_col].map('{:.4f}'.format)

print("\n\n=======================================================")
print("               Model Comparison Summary")
print("=======================================================")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
display(summary_display)

### Compare results across models/ensembles

In [ ]:
# Look at the trends of image quality and uncrtainty metrics over ensemble size
import pandas as pd

results_df = pd.read_csv('MCdropout_30_summary_val.csv')

# Now plot the trends of image quality and uncertainty metrics over ensemble size
import matplotlib.pyplot as plt

# Note that columns come in pairs: mean and std like this "mean_ssim", "mean_ssim.1" and the second one is the std
# and we also need to skip the first row after the headers
columns = results_df.columns[1:]  # Skip the first column which is 'model_name'
# delete the first row after the headers
results_df = results_df.iloc[1:]  # Skip the first row after the headers
ensemble_sizes = results_df['model_name'].str.extract(r'\((\d+)\)').astype(int).values.flatten()
# Plot each metric on a separate plot
# Group columns by metric type
ssim_cols = [col for col in columns if 'ssim' in col and not col.endswith('.1')]
psnr_cols = [col for col in columns if 'psnr' in col and not col.endswith('.1')]
mae_cols = [col for col in columns if 'mae' in col and not col.endswith('.1')]
mse_cols = [col for col in columns if 'mse' in col and not col.endswith('.1')]
ause_cols = [col for col in columns if 'ause' in col and not col.endswith('.1')]
corr_cols = [col for col in columns if 'corr' in col and not col.endswith('.1')]

def plot_metrics(metric_cols, title):
    plt.figure(figsize=(8, 5))
    for col in metric_cols:
        metric_mean = results_df[col].astype(float)
        metric_std = results_df[f"{col}.1"].astype(float)
        plt.errorbar(ensemble_sizes, metric_mean, yerr=metric_std, label=col, fmt='-o', capsize=5)
    plt.title(title)
    plt.xlabel('Ensemble Size')
    plt.xticks(ensemble_sizes)
    plt.legend()
    plt.grid(True)
    plt.show()

# Plot grouped metrics
if ssim_cols:
    plot_metrics(ssim_cols, 'SSIM Metrics Over Ensemble Size')
if psnr_cols:
    plot_metrics(psnr_cols, 'PSNR Metrics Over Ensemble Size')
if mae_cols:
    plot_metrics(mae_cols, 'MAE Metrics Over Ensemble Size')
if mse_cols:
    plot_metrics(mse_cols, 'MSE Metrics Over Ensemble Size')
if ause_cols:
    plot_metrics(ause_cols, 'AUSE Metrics Over Ensemble Size')
if corr_cols:
    plot_metrics(corr_cols, 'Correlation Metrics Over Ensemble Size')

In [ ]:
import matplotlib.pyplot as plt

# List of CSVs to compare
csv_files = [
    # 'MCdropout_15_summary_val.csv',
    'MCdropout_30_summary_val.csv',
    # 'MCdropout_50_summary_val.csv',
    'ensemble_summary_val.csv',
    # 'MCdropout_30_summary_test.csv',
    # 'ensemble_summary_test.csv',
    # Add more CSV file paths here
]

results_dfs = [pd.read_csv(csv) for csv in csv_files]
labels = [csv.split('.')[0] for csv in csv_files]
colors = plt.cm.tab10.colors  # Up to 10 distinct colors

def get_metric_cols(columns, metric):
    return [col for col in columns if metric in col and not col.endswith('.1')]

metrics = ['ssim', 'psnr', 'mae', 'mse', 'ause', 'corr']
metric_titles = {
    'ssim': 'SSIM Metrics Over Ensemble Size',
    'psnr': 'PSNR Metrics Over Ensemble Size',
    'mae': 'MAE Metrics Over Ensemble Size',
    'mse': 'MSE Metrics Over Ensemble Size',
    'ause': 'AUSE Metrics Over Ensemble Size',
    'corr': 'Correlation Metrics Over Ensemble Size'
}

for metric in metrics:
    plt.figure(figsize=(8, 5))
    # Find the minimum max ensemble size across all CSVs
    max_x = min([
        results_df.iloc[1:]['model_name'].str.extract(r'\((\d+)\)').astype(int).values.flatten().max()
        for results_df in results_dfs
    ])
    for i, results_df in enumerate(results_dfs):
        columns = results_df.columns[1:]
        results_df = results_df.iloc[1:]
        ensemble_sizes = results_df['model_name'].str.extract(r'\((\d+)\)').astype(int).values.flatten()
        metric_cols = get_metric_cols(columns, metric)
        for j, col in enumerate(metric_cols):
            metric_mean = results_df[col].astype(float).values
            # Sort by ensemble size
            sorted_idx = np.argsort(ensemble_sizes)
            sorted_ensemble_sizes = ensemble_sizes[sorted_idx]
            sorted_metric_mean = metric_mean[sorted_idx]
            linestyle = '-' if j == 0 else '--'
            label = f"{labels[i]}: {col}"
            # Only plot up to max_x
            mask = sorted_ensemble_sizes <= max_x
            plt.plot(
                sorted_ensemble_sizes[mask], sorted_metric_mean[mask],
                label=label,
                color=colors[i % len(colors)], linestyle=linestyle, marker='o'
            )
    plt.title(metric_titles[metric])
    plt.xlabel('Ensemble Size')
    plt.xlim(left=None, right=max_x)
    plt.xticks(np.arange(sorted_ensemble_sizes.min(), max_x + 1))
    plt.legend()
    plt.grid(True)
    plt.show()


## Post-calibration processing & results

In [ ]:
### Calibration Analysis (Train on Validation, Evaluate on Test)
# This cell performs a separate analysis focused on uncertainty calibration.
# 1. It trains calibration models (Platt, Isotonic) for each model on the VALIDATION data.
# 2. It then evaluates the uncalibrated and calibrated uncertainties on the TEST data.

# This list will store dictionaries of post-calibration results
calibration_results = []

# Load the scan lists for validation and testing, assuming they don't change
all_scans, _ = read_scans_agg_file(SCANS_AGG_FILE)
validation_scans = all_scans['VALIDATION']
test_scans = all_scans['TEST']

for model_config in MODELS_TO_ANALYZE:
    model_name = model_config['name']
    domain = model_config['domain']
    
    # ======================================================================
    # 1. CALIBRATION TRAINING on the VALIDATION set
    # ======================================================================
    print(f"\n--- Training calibration for model: {model_name} ---")
    
    all_val_errors = []
    all_val_uncertainties = []

    for scan_info in tqdm(validation_scans, desc=f"Gathering validation data for {model_name}", leave=False):
        # Load GT and calculate uncalibrated predictions for this validation scan
        gt_volume = load_ground_truth(FILES, scan_info, domain)
        # The 'calculate_volume_metrics_2_pass' is efficient for getting mean and std dev
        _, mean_pred_vol, uncertainty_map_vol = calculate_volume_metrics_2_pass(
            FILES, model_config, scan_info, gt_volume.to(DEVICE), DEVICE
        )
        
        # Move results to CPU and convert to numpy for calibration training
        gt_volume_np = gt_volume.cpu().numpy()
        del gt_volume
        mean_pred_vol_np = mean_pred_vol.cpu().numpy()
        del mean_pred_vol
        uncertainty_map_vol_np = uncertainty_map_vol.cpu().numpy()
        del uncertainty_map_vol

        errors_vol = np.abs(gt_volume_np - mean_pred_vol_np)
        
        all_val_errors.append(errors_vol.flatten())
        all_val_uncertainties.append(uncertainty_map_vol_np.flatten())

    # Consolidate all validation data into single arrays
    val_errors_full = np.concatenate(all_val_errors)
    val_uncertainties_full = np.concatenate(all_val_uncertainties)
    
    # Train the calibration models using the full validation dataset
    print("Calculating Platt Scaler...")
    platt_scaler = calculate_platt_scaler(val_errors_full, val_uncertainties_full)
    print(f"Calibration complete for {model_name}. Platt Scaler T={platt_scaler:.4f}")
    print("Training Isotonic Regression model...")
    isotonic_model = train_isotonic_regression(val_uncertainties_full, val_errors_full, DEVICE)
    print("Done.")
        
    # Clean up memory from the training phase
    del all_val_errors, all_val_uncertainties, val_errors_full, val_uncertainties_full
    gc.collect()

    # ======================================================================
    # 2. FINAL EVALUATION on the TEST set
    # ======================================================================
    print(f"\n--- Evaluating model: {model_name} on the TEST set ---")
    
    for scan_info in tqdm(test_scans, desc=f"Analyzing Test Scans for {model_name}"):
        patient, scan, _ = scan_info
        scan_name = f"p{patient}_{scan}"
        
        # --- Data Loading & Base Uncalibrated Calculation ---
        gt_volume = load_ground_truth(FILES, scan_info, domain)
        gt_volume_np = gt_volume.cpu().numpy()
        
        _, mean_pred_vol, uncal_uncertainty_map = calculate_volume_metrics_2_pass(
            FILES, model_config, scan_info, gt_volume.to(DEVICE), DEVICE
        )
        del gt_volume
        
        mean_pred_vol_np = mean_pred_vol.cpu().numpy()
        del mean_pred_vol
        uncal_uncertainty_map_np = uncal_uncertainty_map.cpu().numpy()
        del uncal_uncertainty_map
        
        # --- Apply Calibrations ---
        print("Applying calibrations...")
        platt_uncertainty_map_np = uncal_uncertainty_map_np * platt_scaler
        iso_uncertainty_map_np = isotonic_model(uncal_uncertainty_map_np)

        # --- Store results for this scan ---
        scan_result = {'model_name': model_name, 'scan_name': scan_name}
        
        # --- Calculate and Store All Metrics (Uncalibrated, Platt, Isotonic) ---
        calibrations = {
            'platt': platt_uncertainty_map_np,
            'iso': iso_uncertainty_map_np
        }

        # We need the error map for AUSE calculation
        errors_vol = np.abs(gt_volume_np - mean_pred_vol_np)
        
        print("Evaluating metrics for each calibration...")
        for cal_name, uncertainty_map in calibrations.items():
            print("Calculating metrics for calibration:", cal_name)

            print("Calculating AUSE...")
            ause_val = calculate_ause_sparsification(uncertainty_map, errors_vol)
            scan_result[f'ause_{cal_name}'] = ause_val

            print("Calculating ECE...")
            for n_bins in [10, 20, 50]:
                eces = calculate_all_eces(gt_volume_np, mean_pred_vol_np, uncertainty_map, n_bins)
                for key, val in eces.items():
                    scan_result[f'{key}_{n_bins}bins_{cal_name}'] = val

            print("Calculating ENCE...")         
            for n_bins in [10, 20, 50]:
                scan_result[f'ence_{n_bins}bins_{cal_name}'] = calculate_ence(gt_volume_np, mean_pred_vol_np, uncertainty_map, DEVICE, n_bins)
            
            print("Calculating NLL...")
            scan_result[f'nll_{cal_name}'] = calculate_nll(gt_volume_np, mean_pred_vol_np, uncertainty_map)
            
            print("Calculating MPIW...")
            mpiws = calculate_mpiw(uncertainty_map, confidence_levels=[0.68, 0.95])
            for key, val in mpiws.items():
                scan_result[f'{key}_{cal_name}'] = val
            print("Done with metrics for", cal_name)

        calibration_results.append(scan_result)

        # Platt Scaling Plots
        print(f"--- Generating combined calibration plots for {model_name} on {scan_name} ---")
        plot_combined_calibration_curves(
            ground_truth=gt_volume_np,
            mean_pred=mean_pred_vol_np,
            platt_uncertainty_map=platt_uncertainty_map_np,
            iso_uncertainty_map=iso_uncertainty_map_np,
            model_name=model_name,
            scan_name=scan_name,
            n_bins=20
        )
        
        # --- Clean up memory ---
        del gt_volume_np, mean_pred_vol_np, uncal_uncertainty_map_np, errors_vol
        del platt_uncertainty_map_np, iso_uncertainty_map_np
        gc.collect()

# Convert results to a pandas DataFrame for easier analysis
calibration_results_df = pd.DataFrame(calibration_results)

print("\n\n✅ Calibration analysis complete.")
calibration_results_df

In [ ]:
### Aggregate Results Summary

# Select only the numeric columns for aggregation
numeric_cols = calibration_results_df.select_dtypes(include=[np.number]).columns
summary = calibration_results_df.groupby('model_name')[numeric_cols].agg(['mean', 'std']).reset_index()

# Save results_df and summary to CSV files
calibration_results_df.to_csv('MCdropout_30_results_calibration.csv', index=False)
summary.to_csv('MCdropout_30_summary_calibration.csv', index=False)

# Prepare a display DataFrame with formatted 'mean ± std' strings
summary_display = pd.DataFrame()
summary_display['model_name'] = summary['model_name']

for col in numeric_cols:
    mean_col = (col, 'mean')
    std_col = (col, 'std')
    # Format the string, handling potential NaN values in std dev
    summary_display[col] = summary[mean_col].map('{:.4f}'.format) + ' ± ' + summary[std_col].map('{:.4f}'.format)

print("\\n\\n=======================================================")
print("               Model Comparison Summary")
print("=======================================================")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
display(summary_display)

In [ ]:
# Example: Combine multiple summary DataFrames from different CSVs
import pandas as pd

# List of summary CSV files to compare
summary_csv_files = [
    'MCdropout_30_summary_calibration.csv',
    'ensemble_summary_calibration.csv',
    # Add more summary CSV file paths here
]
columns_to_include = [
    # 'ece_weighted_abs_20bins_platt',
    'ece_unweighted_abs_20bins_platt',
    'ence_20bins_platt',
    'nll_platt',
    # 'mpiw_68_platt',
    'mpiw_95_platt',
    # 'ece_weighted_abs_20bins_iso',
    'ece_unweighted_abs_20bins_iso',
    'ence_20bins_iso',
    'nll_iso',
    # 'mpiw_68_iso',
    'mpiw_95_iso',
]

# Load each summary CSV as a DataFrame and append to a list
summaries = [pd.read_csv(csv) for csv in summary_csv_files]

for i in range(len(summaries)):
    # Discard the first non-header row
    summaries[i] = summaries[i].iloc[1:]

    # Convert all columns to numeric (except 'model_name')
    for col in summaries[i].columns:
        if col != 'model_name':
            summaries[i][col] = pd.to_numeric(summaries[i][col], errors='raise')

# Concatenate all summary DataFrames row-wise
if summaries:
    combined_summary = pd.concat(summaries, ignore_index=True)
    print("\n\n=======================================================")
    print("         Combined Model Calibration Summaries")
    print("=======================================================")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)
    # Display the dataframe with values formatted as 'mean ± std'
    for col in columns_to_include:
        mean_col = col
        std_col = col + '.1'
        combined_summary[col] = combined_summary[mean_col].map('{:.4f}'.format) + ' ± ' + combined_summary[std_col].map('{:.4f}'.format)
    
    # Keep only the relevant columns
    combined_summary = combined_summary[['model_name'] + columns_to_include]
    display(combined_summary)
else:
    print("No summary CSV files provided.")